# Fine-tunning SFM Base Patch16 on Parihaka Dataset

## Imports

In [1]:
from common import get_data_module, get_trainer_pipeline
import torch
from minerva.models.nets.image.vit import SFM_BasePatch16_Downstream
from functools import partial
from minerva.models.loaders import FromPretrained

/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:54: UserWarning: Reliance on distutils from stdlib is deprecated. Users must rely on setuptools to provide the distutils module. Avoid importing distutils or import setuptools first, and avoid setting SETUPTOOLS_USE_DISTUTILS=stdlib. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Variaveis

In [2]:
root_data_dir = "/workspaces/HIAAC-KR-Dev-Container/shared_data/seam_ai_datasets/seam_ai/images"
root_annotation_dir = "/workspaces/HIAAC-KR-Dev-Container/shared_data/seam_ai_datasets/seam_ai/annotations"
img_size = (512, 512)          # Change this to the size of the images in the dataset
model_name = "sfm-base-patch16"       # Model name (just identifier)
dataset_name = "seam_ai"        # Dataset name (just identifier)
single_channel = True          # If True, the model will be trained with single channel images (instead of 3 channels)

log_dir = "./logs"              # Directory to save logs
batch_size = 1                  # Batch size    
seed = 42                       # Seed for reproducibility
num_epochs = 100                # Number of epochs to train
is_debug = True                 # If True, only 3 batch will be processed for 3 epochs
accelerator = "gpu"             # CPU or GPU
devices = 1                     # Num GPUs

## Data Module

In [3]:
data_module = get_data_module(
    root_data_dir=root_data_dir,
    root_annotation_dir=root_annotation_dir,
    img_size=img_size,
    batch_size=batch_size,
    seed=seed,
    single_channel=single_channel, 
)

data_module

DataModule
    Data: /workspaces/HIAAC-KR-Dev-Container/shared_data/seam_ai_datasets/seam_ai/images
    Annotations: /workspaces/HIAAC-KR-Dev-Container/shared_data/seam_ai_datasets/seam_ai/annotations
    Batch size: 1

In [4]:
# Just to check if the data module is working
data_module.setup("fit")
train_batch_x, train_batch_y = next(iter(data_module.train_dataloader()))
train_batch_x.shape, train_batch_y.shape

(torch.Size([1, 1, 512, 512]), torch.Size([1, 1, 512, 512]))

## **** Create and Load model HERE ****

In [5]:
model = SFM_BasePatch16_Downstream(
    img_size=(512, 512),
    num_classes=6,
    in_chans=1
)

model

SFM_BasePatch16_Downstream(
  (backbone): VisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(1, 768, kernel_size=(16, 16), stride=(16, 16))
      (norm): Identity()
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (patch_drop): Identity()
    (norm_pre): Identity()
    (blocks): Sequential(
      (0): Block(
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=768, out_features=2304, bias=True)
          (q_norm): Identity()
          (k_norm): Identity()
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=768, out_features=768, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (ls1): Identity()
        (drop_path1): Identity()
        (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (act): GELU(ap

In [6]:
model = FromPretrained(
    model=model,
    ckpt_path="/workspaces/HIAAC-KR-Dev-Container/shared_data/seismic_foundation_model/pretrained_models/SFM-Base-512.pth",
    ckpt_key="model",
    strict=False,
    filter_keys=["^blocks*", "^cls_token", "^pos_embed", "^patch_embed", "^norm"],
    keys_to_rename={"": "backbone."},
    ckpt_load_weights_only=False,
    error_on_missing_keys=False
)


Performing key renaming with: {'': 'backbone.'}
	Renaming key: cls_token -> backbone.cls_token (changed: True)
	Renaming key: pos_embed -> backbone.pos_embed (changed: True)
	Renaming key: patch_embed.proj.weight -> backbone.patch_embed.proj.weight (changed: True)
	Renaming key: patch_embed.proj.bias -> backbone.patch_embed.proj.bias (changed: True)
	Renaming key: blocks.0.norm1.weight -> backbone.blocks.0.norm1.weight (changed: True)
	Renaming key: blocks.0.norm1.bias -> backbone.blocks.0.norm1.bias (changed: True)
	Renaming key: blocks.0.attn.qkv.weight -> backbone.blocks.0.attn.qkv.weight (changed: True)
	Renaming key: blocks.0.attn.qkv.bias -> backbone.blocks.0.attn.qkv.bias (changed: True)
	Renaming key: blocks.0.attn.proj.weight -> backbone.blocks.0.attn.proj.weight (changed: True)
	Renaming key: blocks.0.attn.proj.bias -> backbone.blocks.0.attn.proj.bias (changed: True)
	Renaming key: blocks.0.norm2.weight -> backbone.blocks.0.norm2.weight (changed: True)
	Renaming key: blocks.0

## Pipeline

In [7]:
pipeline = get_trainer_pipeline(
    model=model,
    model_name=model_name,
    dataset_name=dataset_name,
    log_dir=log_dir,
    num_epochs=num_epochs,
    accelerator=accelerator,
    devices=devices,
    is_debug=is_debug,
    seed=seed,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Seed set to 42


Log directory set to: /workspaces/HIAAC-KR-Dev-Container/Minerva-Dev/docs/notebooks/examples/seismic/facies_classification/parihaka/logs/sfm-base-patch16/seam_ai


In [8]:
pipeline.run(data_module, task="fit")

/usr/local/lib/python3.10/dist-packages/lightning/fabric/loggers/csv_logs.py:268: Experiment logs directory ./logs/sfm-base-patch16/seam_ai exists and is not empty. Previous log files in this directory will be deleted when the new ones are saved!


Pipeline info saved at: /workspaces/HIAAC-KR-Dev-Container/Minerva-Dev/docs/notebooks/examples/seismic/facies_classification/parihaka/logs/sfm-base-patch16/seam_ai/run_2024-12-06-14-24-03cc37a68a1cb74db7b8866e7e11eb966d.yaml


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory ./logs/sfm-base-patch16/seam_ai/checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type              | Params | Mode 
-------------------------------------------------------
0 | backbone | VisionTransformer | 90.2 M | train
1 | fc       | Identity          | 0      | train
2 | loss_fn  | CrossEntropyLoss  | 0      | train
-------------------------------------------------------
90.2 M    Trainable params
0         Non-trainable params
90.2 M    Total params
360.821   Total estimated model params size (MB)
301       Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/loops/fit_loop.py:298: The number of training batches (3) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 2: 100%|██████████| 3/3 [00:00<00:00,  4.39it/s, v_num=m_ai, val_loss=3.670, train_loss=1.320]

`Trainer.fit` stopped: `max_epochs=3` reached.


Epoch 2: 100%|██████████| 3/3 [00:45<00:00,  0.07it/s, v_num=m_ai, val_loss=3.670, train_loss=1.320]
Pipeline info saved at: /workspaces/HIAAC-KR-Dev-Container/Minerva-Dev/docs/notebooks/examples/seismic/facies_classification/parihaka/logs/sfm-base-patch16/seam_ai/run_2024-12-06-14-24-03cc37a68a1cb74db7b8866e7e11eb966d.yaml


In [9]:
print(f"Checkpoint saved at {pipeline.trainer.checkpoint_callback.last_model_path}")

Checkpoint saved at ./logs/sfm-base-patch16/seam_ai/checkpoints/last-v2.ckpt
